# Airline Commercial Controlling & BI Project

### 1) Project Goal:
This project analyzes airline commercial performance with a focus on route profitability, cost efficiency, and margin stability.
Using operational, revenue, and cost data, the analysis calculates key airline KPIs (RASK, CASK, Load Factor, Yield, Contribution Margin) and translates them into actionable insights for pricing, capacity, and network decisions.
The final output supports data-driven commercial decision-making through structured analysis and BI-ready visualizations.



## 2) Business Questions & Analytical Scope

Q1 ) Which routes are currently operating below the target net profit margin of 3.9%, and what pricing, capacity, or cost levers could improve their profitability?

Q2) How would a 2.4% reduction in jet fuel prices impact route-level and network-level contribution margins, and to what extent can this offset rising non-fuel costs such as labour expenses?

Q3) Which routes and passenger segments show the highest potential for ancillary revenue growth, and how can non-ticket revenues support overall route profitability?

Q4) How does the passenger load factor correlate with route profitability, and are there capacity-constrained routes where additional frequency or up-gauging could improve revenue and margins?

### 3) Core KPIs


| Revenue KPIs | Cost KPIs | Profitability KPIs |
| :--- | :--- | :--- |
| 0) Passenger Revenue | Fuel Cost | Contribution Margin I |
| 1) Average Fare | Airport & Handling Cost | Contribution Margin II |
| 2) Load Factor | Total Cost | Route Profit / Loss |
| 3) RASK | CASK | Break-Even Load Factor |


### 4) Data Selection & Assumptions

| Time Scope | Network Scope | Cost Structure | Revenue Assumptions | Data Source Policy |
| :--- | :--- | :--- | :--- | :--- |
| Monthly data for 2026 | 10–15 routes | Fuel: 25–25.7% | Stable / slightly softening yields | Public datasets only |
| Enables seasonality, trend, and margin analysis | Hub-and-spoke model | Labour: 28% | Ancillary revenue growth +5.5% | Industry benchmarks (IATA-style) |
| | Hub selected: Frankfurt (FRA) | Other costs: 47% | Clear split between ticket and ancillary revenue | |
| | | Maintenance & ownership uplift included | | |


# Import Libraries 

In [352]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load OpenFlights Data

In [353]:
airports = pd.read_csv("airports.csv")
routes = pd.read_csv("routes.csv")
airlines = pd.read_csv("airlines.csv")

In [354]:
print(airports.shape)
print(routes.shape)
print(airlines.shape)

(4263, 9)
(33322, 5)
(618, 6)


In [355]:
airports.head()


,Unnamed: 0,name,latitude,longitude,altitude,country,icao,iata,score
0,0,London Heathrow Airport,51.47,-0.46,83.00,GB,EGLL,LHR,1251675
1,1,Los Angeles International Airport,33.94,-118.41,125.00,US,KLAX,LAX,1335475
2,2,Chicago O'Hare International Airport,41.98,-87.90,680.00,US,KORD,ORD,1503175
3,3,John F Kennedy International Airport,40.64,-73.78,13.00,US,KJFK,JFK,1052075
4,4,Hartsfield Jackson Atlanta International Airport,33.64,-84.43,1026.00,US,KATL,ATL,2002475


In [356]:
routes.head()

,Unnamed: 0,airline,dep,arr,equipment
0,2,#C7,MAO,CIZ,EMB
1,3,2A,DAC,CGP,AT7
2,4,2A,DAC,CXB,AT7
3,5,2A,DAC,SPD,AT7
4,6,2G,IKT,BTK,A81


In [357]:
airlines.head()

,Unnamed: 0,name,iata,icao,callsign,country
0,3,1Time Airline,1T,RNX,NEXTIME,South Africa
1,10,40-Mile Air,Q5,MLA,MILE-AIR,United States
2,13,Ansett Australia,AN,AAA,ANSETT,Australia
3,21,Aigle Azur,ZI,AAF,AIGLE AZUR,France
4,22,Aloha Airlines,AQ,AAH,ALOHA,United States


## Drop unnecessary index column

In [358]:
airports = airports.drop(columns=["Unnamed: 0"])
airlines = airlines.drop(columns=["Unnamed: 0"])
routes = routes.drop(columns=["Unnamed: 0"])





In [359]:
airports.info()
airlines.info()
routes.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4263 entries, 0 to 4262
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       4263 non-null   object 
 1   latitude   4263 non-null   float64
 2   longitude  4263 non-null   float64
 3   altitude   4229 non-null   float64
 4   country    4251 non-null   object 
 5   icao       4263 non-null   object 
 6   iata       4263 non-null   object 
 7   score      4263 non-null   int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 266.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618 entries, 0 to 617
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      618 non-null    object
 1   iata      618 non-null    object
 2   icao      618 non-null    object
 3   callsign  618 non-null    object
 4   country   618 non-null    object
dtypes: object(5)
memory usage: 24.3+ KB
<class 'pandas.core

The airport, airline, and route tables exhibit consistent structures and appropriate data types for commercial and network analysis. Key identifier fields are complete and reliable, supporting downstream joins and KPI calculations. Overall, the data quality is sufficient to proceed with cleaning, standardization, and analytical modeling.



## Lufthansa Route Subset

Lufthansa routes are identified using the airline IATA code (`LH`).

In [360]:
lh_routes = routes[routes["airline"] == "LH"]

In [361]:
lh_routes.shape


(236, 4)

## Hub Filtering

Frankfurt (FRA) is treated as Lufthansa’s primary hub for network analysis.

In [362]:
lh_fra_routes = lh_routes[
    (lh_routes["dep"] == "FRA") |
    (lh_routes["arr"] == "FRA")
]


In [363]:
lh_fra_routes.shape


(137, 4)

## Airport Reference Data

Only fields required for geographic and country-level analysis are retained.

In [364]:
airports_clean = airports[[
    "iata",
    "latitude",
    "longitude",
    "country"
]].copy()

airports_clean = airports_clean.rename(columns={
    "iata": "airport_code",
    "latitude": "lat",
    "longitude": "lon",
    "country": "country"
})

airports_clean.head()


,airport_code,lat,lon,country
0,LHR,51.47,-0.46,GB
1,LAX,33.94,-118.41,US
2,ORD,41.98,-87.90,US
3,JFK,40.64,-73.78,US
4,ATL,33.64,-84.43,US


## Route Enrichment – Departure Airport
Departure airport geographic and country attributes are merged into the route dataset.

In [365]:
lh_fra_routes = lh_fra_routes.merge(
    airports_clean,
    left_on="dep",
    right_on="airport_code",
    how="left"
)

lh_fra_routes = lh_fra_routes.rename(columns={
    "lat": "dep_lat",
    "lon": "dep_lon",
    "country": "dep_country"
})

lh_fra_routes.drop(columns=["airport_code"], inplace=True)


## Route Enrichment – Arrival Airport
Arrival airport geographic and country attributes are merged into the route dataset.


In [366]:
lh_fra_routes = lh_fra_routes.merge(
    airports_clean,
    left_on="arr",
    right_on="airport_code",
    how="left"
)

lh_fra_routes = lh_fra_routes.rename(columns={
    "lat": "arr_lat",
    "lon": "arr_lon",
    "country": "arr_country"
})

lh_fra_routes.drop(columns=["airport_code"], inplace=True)


In [367]:
lh_fra_routes.isna().sum()


airline        0
dep            0
arr            0
equipment      0
dep_lat        0
dep_lon        0
dep_country    0
arr_lat        0
arr_lon        0
arr_country    0
dtype: int64

The route enrichment step results in complete geographic coverage for both departure and arrival airports, confirming successful joins.


In [368]:
lh_fra_routes.head()


,airline,dep,arr,equipment,dep_lat,dep_lon,dep_country,arr_lat,arr_lon,arr_country
0,LH,FRA,ABV,343,50.03,8.56,DE,9.01,7.26,NG
1,LH,FRA,AGP,321,50.03,8.56,DE,36.67,-4.50,ES
2,LH,FRA,ALA,333,50.03,8.56,DE,43.35,77.04,KZ
3,LH,FRA,ALC,321,50.03,8.56,DE,38.28,-0.56,ES
4,LH,FRA,ALG,319 320 321,50.03,8.56,DE,36.69,3.21,DZ


## Route Distance Calculation
Route distance is calculated as a foundation for capacity, cost, and revenue KPIs.


In [369]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c


### Apply distance calculation

In [370]:
lh_fra_routes["distance_km"] = haversine(
    lh_fra_routes["dep_lat"],
    lh_fra_routes["dep_lon"],
    lh_fra_routes["arr_lat"],
    lh_fra_routes["arr_lon"]
)


In [371]:
lh_fra_routes[["dep", "arr", "distance_km"]].head()


,dep,arr,distance_km
0,FRA,ABV,4563.18
1,FRA,AGP,1816.47
2,FRA,ALA,5090.14
3,FRA,ALC,1492.67
4,FRA,ALG,1543.61


## Route Classification
Routes are classified into haul categories based on distance, reflecting commercial planning practice.


In [372]:
def route_type(distance):
    if distance < 1500:
        return "Short-haul"
    elif distance < 4000:
        return "Medium-haul"
    else:
        return "Long-haul"

lh_fra_routes["route_type"] = lh_fra_routes["distance_km"].apply(route_type)


## Capacity Assumptions
At portfolio level, capacity is modeled using haul-based average seat assumptions rather than aircraft-level assignments.


In [373]:
def avg_seats(route_type):
    if route_type == "Short-haul":
        return 180   # A320 family
    elif route_type == "Medium-haul":
        return 250   # A321 / A321LR
    else:
        return 300   # A330 / A350 / B787

lh_fra_routes["avg_seats"] = lh_fra_routes["route_type"].apply(avg_seats)


Monthly frequencies are assigned based on typical network planning patterns by haul type.


In [374]:
def monthly_flights(route_type):
    if route_type == "Short-haul":
        return 120   # ~4 flights/day
    elif route_type == "Medium-haul":
        return 60    # ~2 flights/day
    else:
        return 30    # ~1 flight/day

lh_fra_routes["monthly_flights"] = lh_fra_routes["route_type"].apply(monthly_flights)


## Demand Assumption
Passenger volumes are estimated using a projected average load factor.


In [375]:
LOAD_FACTOR_2026 = 0.838

lh_fra_routes["passengers_per_month"] = (
    lh_fra_routes["avg_seats"] *
    lh_fra_routes["monthly_flights"] *
    LOAD_FACTOR_2026
)


## Available Seat Kilometers (ASK)
ASK is a core capacity metric in airline commercial controlling.


In [376]:
lh_fra_routes["ASK"] = (
    lh_fra_routes["avg_seats"] *
    lh_fra_routes["monthly_flights"] *
    lh_fra_routes["distance_km"]
)


In [377]:
lh_fra_routes[[
    "dep", "arr",
    "route_type",
    "avg_seats",
    "monthly_flights",
    "passengers_per_month",
    "ASK"
]].head()


,dep,arr,route_type,avg_seats,monthly_flights,passengers_per_month,ASK
0,FRA,ABV,Long-haul,300,30,7542.00,41068615.09
1,FRA,AGP,Medium-haul,250,60,12570.00,27247046.63
2,FRA,ALA,Long-haul,300,30,7542.00,45811266.78
3,FRA,ALC,Short-haul,180,120,18100.80,32241729.68
4,FRA,ALG,Medium-haul,250,60,12570.00,23154195.79


## Revenue & Unit Revenue (RASK) Simulation

This section estimates route-level revenue and unit revenue to support commercial performance assessment and route prioritization.

### Yield Assumptions
Passenger yield is modeled by haul type, reflecting standard commercial planning practice.


In [378]:
def passenger_yield(route_type):
    if route_type == "Short-haul":
        return 0.11   # €/km
    elif route_type == "Medium-haul":
        return 0.085
    else:
        return 0.065

lh_fra_routes["yield_per_km"] = lh_fra_routes["route_type"].apply(passenger_yield)


## Ticket Revenue

In [379]:
lh_fra_routes["ticket_revenue"] = (
    lh_fra_routes["passengers_per_month"] *
    lh_fra_routes["distance_km"] *
    lh_fra_routes["yield_per_km"]
)


## Ancillary Revenue
Ancillary revenue is modeled by haul type, reflecting higher penetration on shorter and more price-sensitive routes.


In [380]:
def ancillary_per_passenger(route_type):
    if route_type == "Short-haul":
        return 22
    elif route_type == "Medium-haul":
        return 28
    else:
        return 35

lh_fra_routes["ancillary_revenue"] = (
    lh_fra_routes["passengers_per_month"] *
    lh_fra_routes["route_type"].apply(ancillary_per_passenger)
)


## Total Revenue & RASK

In [381]:
lh_fra_routes["total_revenue"] = (
    lh_fra_routes["ticket_revenue"] +
    lh_fra_routes["ancillary_revenue"]
)

lh_fra_routes["RASK"] = lh_fra_routes["total_revenue"] / lh_fra_routes["ASK"]


In [382]:
lh_fra_routes[[
    "dep", "arr",
    "route_type",
    "ticket_revenue",
    "ancillary_revenue",
    "total_revenue",
    "RASK"
]].head()


,dep,arr,route_type,ticket_revenue,ancillary_revenue,total_revenue,RASK
0,FRA,ABV,Long-haul,2237007.46,263970.00,2500977.46,0.06
1,FRA,AGP,Medium-haul,1940807.13,351960.00,2292767.13,0.08
2,FRA,ALA,Long-haul,2495339.70,263970.00,2759309.70,0.06
3,FRA,ALC,Short-haul,2972042.64,398217.60,3370260.24,0.10
4,FRA,ALG,Medium-haul,1649273.37,351960.00,2001233.37,0.09


## Cost Structure, CASK & Contribution Margin (2026)
This section converts route-level revenue into profitability using 2026 cost assumptions, calculating CASK, contribution margin, and route-level profit per passenger.

###  Define 2026 cost assumptions

In [383]:
FUEL_SHARE = 0.257
LABOUR_SHARE = 0.28
OTHER_SHARE = 1 - (FUEL_SHARE + LABOUR_SHARE)

COST_PER_ASK = {
    "Short-haul": 0.085,
    "Medium-haul": 0.075,
    "Long-haul": 0.065
}


CASK decreases with distance due to fixed-cost dilution despite higher absolute operating costs.

### Calculate Total Operating Cost

In [384]:
lh_fra_routes["CASK"] = lh_fra_routes["route_type"].map(COST_PER_ASK)

lh_fra_routes["total_cost"] = (
    lh_fra_routes["ASK"] *
    lh_fra_routes["CASK"]
)


### Cost breakdown (Fuel / Labour / Other)

In [385]:
lh_fra_routes["fuel_cost"] = lh_fra_routes["total_cost"] * FUEL_SHARE
lh_fra_routes["labour_cost"] = lh_fra_routes["total_cost"] * LABOUR_SHARE
lh_fra_routes["other_cost"] = lh_fra_routes["total_cost"] * OTHER_SHARE


###  Contribution Margin

In [386]:
lh_fra_routes["contribution_margin"] = (
    lh_fra_routes["total_revenue"] -
    lh_fra_routes["total_cost"]
)

lh_fra_routes["margin_pct"] = (
    lh_fra_routes["contribution_margin"] /
    lh_fra_routes["total_revenue"]
)


###  Net Profit per Passenger

In [387]:
lh_fra_routes["profit_per_passenger"] = (
    lh_fra_routes["contribution_margin"] /
    lh_fra_routes["passengers_per_month"]
)


In [388]:
lh_fra_routes[[
    "dep", "arr",
    "route_type",
    "total_revenue",
    "total_cost",
    "contribution_margin",
    "margin_pct",
    "profit_per_passenger"
]].head()


,dep,arr,route_type,total_revenue,total_cost,contribution_margin,margin_pct,profit_per_passenger
0,FRA,ABV,Long-haul,2500977.46,2669459.98,-168482.52,-0.07,-22.34
1,FRA,AGP,Medium-haul,2292767.13,2043528.50,249238.63,0.11,19.83
2,FRA,ALA,Long-haul,2759309.70,2977732.34,-218422.64,-0.08,-28.96
3,FRA,ALC,Short-haul,3370260.24,2740547.02,629713.22,0.19,34.79
4,FRA,ALG,Medium-haul,2001233.37,1736564.68,264668.68,0.13,21.06


# Route-Level Profitability (Baseline Scenario)

The table above summarizes financial performance for the Frankfurt-based network, integrating revenue, cost, and contribution margin calculations.

### **Key Insights**

* **Long-haul routes** show structural margin pressure. High revenues are offset by operating costs, resulting in negative margins and net losses per passenger.

* **Medium-haul routes** achieve sustainable profitability (10–13% margin), benefiting from balanced yield, efficient aircraft utilization, and flexible frequency.

* **Short-haul leisure routes** act as cash generators, with strong margins and high profit per passenger due to favorable RASK–CASK spread and lower costs.

**Executive Takeaways**

* Short- and medium-haul routes effectively subsidize loss-making long-haul operations.

* Profitability is driven by the **RASK–CASK spread** rather than absolute revenue levels.

* Route-level contribution margin analysis is critical for network optimization and capacity allocation.

* Sustained profitability on long-haul routes may require pricing adjustments, capacity optimization, or cost restructuring.

## Scenario & Sensitivity Analysis: Jet Fuel Price Decrease (–2.4%)

We evaluate the impact of a 2.4% decline in jet fuel prices on route-level profitability, with a focus on long-haul routes.


In [389]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', '__session__', '_i', '_ii', '_iii', '_i1', 'pd', 'np', 'plt', 'sns', '_i2', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', 'airports', 'routes', 'airlines', '_i10', '_i11', '_11', '_i12', '_12', '_i13', '_13', '_i14', '_i15', '_15', '_i16', 'lh_routes', '_i17', '_17', '_i18', 'lh_fra_routes', '_18', '_i19', '_19', '_i20', 'airports_clean', '_20', '_i21', '_i22', '_i23', '_23', '_i24', '_24', '_i25', '_25', '_i26', 'haversine', '_i27', '_i28', '_28', '_i29', 'route_type', '_i30', 'avg_seats', '_i31', 'monthly_flights', '_i32', 'LOAD_FACTOR_2026', '_i33', '_i34', '_34', '_i35', 'passenger_yield', '_i36', '_i37', 'ancillary_per_passenger', '_i38', '_i39', '_39', '_i40', 'FUEL_SHARE', 'LABOUR_SHARE', 'OTHER_SHARE', 'COST_PER_ASK', '_i41', '_i42', '_i43', '_i44', '_i45', '_45', '_i46', '_46', '_

In [390]:
df = lh_fra_routes.copy() 

In [391]:
df.shape 

(137, 29)

In [392]:
df.columns

Index(['airline', 'dep', 'arr', 'equipment', 'dep_lat', 'dep_lon',
       'dep_country', 'arr_lat', 'arr_lon', 'arr_country', 'distance_km',
       'route_type', 'avg_seats', 'monthly_flights', 'passengers_per_month',
       'ASK', 'yield_per_km', 'ticket_revenue', 'ancillary_revenue',
       'total_revenue', 'RASK', 'CASK', 'total_cost', 'fuel_cost',
       'labour_cost', 'other_cost', 'contribution_margin', 'margin_pct',
       'profit_per_passenger'],
      dtype='object')

In [393]:
df[['route_type', 'total_cost', 'contribution_margin', 'margin_pct']].describe()

,total_cost,contribution_margin,margin_pct
count,137.00,137.00,137.00
mean,2616860.44,157232.49,0.13
std,1560975.80,453499.24,0.20
min,287395.92,-826057.08,-0.14
25%,1424659.86,-334663.54,-0.10
50%,2246499.62,427479.26,0.19
75%,3788899.52,524269.29,0.27
max,6728562.20,629713.22,0.60


### Define fuel scenario assumptions

In [394]:
FUEL_PRICE_CHANGE = -0.024  # -2.4% fuel price forecast

# Copy baseline data
df = lh_fra_routes.copy()

# Baseline snapshot
df[['route_type', 'total_cost', 'contribution_margin', 'margin_pct']].describe()


,total_cost,contribution_margin,margin_pct
count,137.00,137.00,137.00
mean,2616860.44,157232.49,0.13
std,1560975.80,453499.24,0.20
min,287395.92,-826057.08,-0.14
25%,1424659.86,-334663.54,-0.10
50%,2246499.62,427479.26,0.19
75%,3788899.52,524269.29,0.27
max,6728562.20,629713.22,0.60


### Split cost into fuel vs non-fuel

In [395]:
df['fuel_cost'] = df['total_cost'] * FUEL_SHARE
df['non_fuel_cost'] = df['total_cost'] * (1 - FUEL_SHARE)


### Apply fuel price decrease

In [396]:
df['fuel_cost_scenario'] = df['fuel_cost'] * (1 + FUEL_PRICE_CHANGE)


### Recalculate total cost (scenario)

In [397]:
df['total_cost_scenario'] = df['fuel_cost_scenario'] + df['non_fuel_cost']


### Recalculate margins (scenario)

In [398]:
df['contribution_margin_scenario'] = (
    df['total_revenue'] - df['total_cost_scenario']
)

df['margin_pct_scenario'] = (
    df['contribution_margin_scenario'] / df['total_revenue']
)


### Compare BEFORE vs AFTER

In [399]:
comparison = df[[
    'dep', 'arr', 'route_type',
    'margin_pct', 'margin_pct_scenario',
    'profit_per_passenger'
]]

comparison


,dep,arr,route_type,margin_pct,margin_pct_scenario,profit_per_passenger
0,FRA,ABV,Long-haul,-0.07,-0.06,-22.34
1,FRA,AGP,Medium-haul,0.11,0.11,19.83
2,FRA,ALA,Long-haul,-0.08,-0.07,-28.96
3,FRA,ALC,Short-haul,0.19,0.19,34.79
4,FRA,ALG,Medium-haul,0.13,0.14,21.06
...,...,...,...,...,...,...
132,FRA,YYZ,Long-haul,-0.10,-0.09,-44.71
133,FRA,WAW,Short-haul,0.25,0.25,29.69
134,FRA,ZAG,Short-haul,0.28,0.28,28.29
135,FRA,ZRH,Short-haul,0.46,0.46,24.45


### Interpretation of Results:

#### Long-haul routes: losses reduce, but remain negative


* Margins improve slightly (e.g., FRA–ABV: –6.7% → –6.1%), but remain negative.

* Fuel relief reduces losses but does not overcome structural costs like aircraft ownership, maintenance, and crew.

**Conclusion:** Long-haul economics remain challenging; cost reductions alone are insufficient.

#### Medium-haul routes:

* Margins improve meaningfully (e.g., FRA–AGP: 10.9% → 11.4%).

* Even small fuel savings translate into material margin gains.

**Conclusion:** Medium-haul routes are resilient and fuel-sensitive; they drive network profitability.

### Short-haul routes: already strong, now even stronger

* Already strong margins improve slightly (e.g., FRA–ALC: 18.7% → 19.2%).

* Low CASK, strong ancillary revenue, and high frequency efficiency make them robust cash generators.

#### Executive Insight
The fuel price reduction scenario demonstrates that cost relief alone is insufficient to restore long-haul route profitability. Despite margin improvements, long-haul routes remain significantly below the Group’s 3.9% profitability benchmark, confirming the structural nature of long-haul margin pressure.

Medium- and short-haul routes benefit disproportionately from fuel cost reductions and continue to act as the primary margin stabilizers within the network. From a capital allocation perspective, future capacity growth should be concentrated on these routes, while long-haul services require either strategic justification or structural intervention through pricing, fleet, or schedule optimization.

## SCENARIO B — Load Factor Improvement

In [400]:
# Step 0: Define scenario load factor

LOAD_FACTOR_SCENARIO = 0.838  # 83.8% target

### Passengers recalculation

In [401]:
df["passengers_scenario_B"] = (
    df["avg_seats"] * df["monthly_flights"] * LOAD_FACTOR_SCENARIO
)


### Passenger yield

In [402]:
df["passenger_yield"] = df["route_type"].apply(passenger_yield)


### Ticket revenue

In [403]:
df["ticket_revenue_B"] = (
    df["passengers_scenario_B"]
    * df["distance_km"]
    * df["yield_per_km"]
)


### Ancillary revenue

In [404]:
df["ancillary_pp"] = df["route_type"].apply(ancillary_per_passenger)
df["ancillary_revenue_B"] = df["passengers_scenario_B"] * df["ancillary_pp"]


### Total revenue & contribution margin

In [405]:
df["total_revenue_B"] = df["ticket_revenue_B"] + df["ancillary_revenue_B"]
df["contribution_margin_B"] = df["total_revenue_B"] - df["total_cost"]


### Margin %

In [406]:
df["margin_pct_B"] = df["contribution_margin_B"] / df["total_revenue_B"]


### Comparison table

In [407]:
scenario_B_results = df[[
    "dep", "arr", "route_type",
    "margin_pct", "margin_pct_B",
    "profit_per_passenger"
]].copy()

scenario_B_results["margin_change_pp"] = (
    scenario_B_results["margin_pct_B"]
    - scenario_B_results["margin_pct"]
)

scenario_B_results.sort_values(
    "margin_change_pp",
    ascending=False
).head(10)


,dep,arr,route_type,margin_pct,margin_pct_B,profit_per_passenger,margin_change_pp
0,FRA,ABV,Long-haul,-0.07,-0.07,-22.34,0.00
94,FRA,NTE,Short-haul,0.26,0.26,28.96,0.00
88,FRA,MXP,Short-haul,0.35,0.35,26.19,0.00
89,FRA,NAP,Short-haul,0.22,0.22,31.51,0.00
90,FRA,NBO,Long-haul,-0.10,-0.10,-44.37,0.00
91,FRA,NCE,Short-haul,0.28,0.28,28.13,0.00
92,FRA,NCL,Short-haul,0.25,0.25,29.61,0.00
93,FRA,NQZ,Long-haul,-0.06,-0.06,-19.15,0.00
95,FRA,NUE,Short-haul,0.55,0.55,23.63,0.00
69,FRA,KTW,Short-haul,0.27,0.27,28.42,0.00


In [408]:
# Check if revenue per passenger is correct
df[['passengers_scenario_B', 'passenger_yield', 'ticket_revenue_B']].head()
df[['passengers_scenario_B', 'ancillary_pp', 'ancillary_revenue_B']].head()

,passengers_scenario_B,ancillary_pp,ancillary_revenue_B
0,7542.00,35,263970.00
1,12570.00,28,351960.00
2,7542.00,35,263970.00
3,18100.80,22,398217.60
4,12570.00,28,351960.00


#### Scenario B – Load Factor Improvement

Improving load factors to the 83.8% network target delivers stronger margin uplift than fuel price relief, particularly on long-haul routes where fixed costs dominate. Higher utilization improves unit economics without increasing capacity or cost, narrowing—but not fully closing—the profitability gap on structurally weak routes. This confirms load factor optimization as a higher-impact lever than fuel price movements for margin recovery.

# SCENARIO C — Ancillary Revenue Upside

#### Define uplift

In [409]:
ANCILLARY_UPLIFT_C = 1.055  # +5.5% ancillary revenue improvement


### Apply uplift

In [410]:
df["ancillary_revenue_C"] = (
    df["ancillary_revenue"] * ANCILLARY_UPLIFT_C
)



### Recalculate total revenue

In [411]:
df["total_revenue_C"] = (
    df["ticket_revenue"] +
    df["ancillary_revenue_C"]
)


### Recalculate contribution margin & margin %

In [412]:
df["contribution_margin_C"] = (
    df["total_revenue_C"] - df["total_cost"]
)

df["margin_pct_C"] = (
    df["contribution_margin_C"] / df["total_revenue_C"]
)


### Executive comparison table

In [413]:
scenario_C_results = df[[
    "dep", "arr", "route_type",
    "margin_pct", "margin_pct_C",
    "profit_per_passenger"
]].copy()

scenario_C_results["margin_change_pp"] = (
    scenario_C_results["margin_pct_C"] -
    scenario_C_results["margin_pct"]
)

scenario_C_results.sort_values(
    "margin_change_pp",
    ascending=False
).head(10)


,dep,arr,route_type,margin_pct,margin_pct_C,profit_per_passenger,margin_change_pp
95,FRA,NUE,Short-haul,0.55,0.56,23.63,0.01
40,FRA,DUS,Short-haul,0.55,0.56,23.62,0.01
79,FRA,LUX,Short-haul,0.57,0.58,23.50,0.01
118,FRA,STR,Short-haul,0.60,0.61,23.34,0.01
25,FRA,BSL,Short-haul,0.46,0.47,24.40,0.01
54,FRA,HAJ,Short-haul,0.46,0.47,24.41,0.01
135,FRA,ZRH,Short-haul,0.46,0.47,24.45,0.01
136,MUC,FRA,Short-haul,0.45,0.46,24.57,0.01
73,FRA,LEJ,Short-haul,0.45,0.46,24.58,0.01
24,FRA,BRU,Short-haul,0.44,0.46,24.61,0.01


## Route-type level summary

In [414]:
scenario_C_summary = (
    scenario_C_results
    .groupby("route_type")[["margin_change_pp"]]
    .mean()
    .reset_index()
)


### Interpretation

**Key insight:**
Ancillary uplift improves margins across all routes, but the impact is incremental rather than transformational. Short-haul routes benefit the most in relative terms, yet the overall margin recovery is materially smaller than that achieved through load factor improvements.

**Controlling takeaway:**
Ancillary revenue is a valuable optimization lever, but it functions best as a margin enhancer, not a primary profitability driver.

#### Scenario C – Ancillary Revenue Upside

A 5.5% increase in ancillary revenue improves margins across all route types, with the strongest impact on short- and medium-haul routes where ancillary penetration is structurally higher. While ancillaries support long-haul economics, they remain insufficient to offset high fixed and ownership costs, confirming their limited role as a primary turnaround lever.

In [415]:
print(df.columns.tolist())


['airline', 'dep', 'arr', 'equipment', 'dep_lat', 'dep_lon', 'dep_country', 'arr_lat', 'arr_lon', 'arr_country', 'distance_km', 'route_type', 'avg_seats', 'monthly_flights', 'passengers_per_month', 'ASK', 'yield_per_km', 'ticket_revenue', 'ancillary_revenue', 'total_revenue', 'RASK', 'CASK', 'total_cost', 'fuel_cost', 'labour_cost', 'other_cost', 'contribution_margin', 'margin_pct', 'profit_per_passenger', 'non_fuel_cost', 'fuel_cost_scenario', 'total_cost_scenario', 'contribution_margin_scenario', 'margin_pct_scenario', 'passengers_scenario_B', 'passenger_yield', 'ticket_revenue_B', 'ancillary_pp', 'ancillary_revenue_B', 'total_revenue_B', 'contribution_margin_B', 'margin_pct_B', 'ancillary_revenue_C', 'total_revenue_C', 'contribution_margin_C', 'margin_pct_C']


In [416]:
# Build dashboard with all scenarios
dashboard_df = df[[
    "dep", "arr", "route_type",
    "margin_pct",          # Baseline
    "margin_pct_scenario", # Fuel price scenario
    "margin_pct_B",        # Load factor scenario
    "margin_pct_C",        # Ancillary scenario
    "profit_per_passenger"
]].copy()


### CSV export

In [424]:
dashboard_df.to_csv(
    r"D:\lufthansa\airline_scenario_dashboard.csv",
    index=False
)


# Airline Commercial Controlling & BI — Executive Summary

**Objective**
This project evaluates route-level profitability across Lufthansa’s Frankfurt hub network by integrating capacity, revenue, and cost economics. The analysis identifies structural margin drivers and tests three commercial levers to support data-driven pricing, capacity, and network decisions.

**Baseline Performance (2026)**

* Short- and medium-haul routes are the network’s margin backbone
* Short-haul routes deliver strong double-digit margins driven by low CASK, high frequency efficiency, and strong ancillary penetration.
* Medium-haul routes achieve stable, sustainable profitability, benefiting from balanced yield and cost dilution.

#### Long-haul routes are structurally loss-making
* Despite high absolute revenues, long-haul services remain below the 3.9% profitability threshold.
* High fixed costs (aircraft ownership, maintenance, crew) constrain margin recovery.

**Key insight:**
Profitability is driven by RASK–CASK spread, not revenue scale.

#### Scenario A — Fuel Price Reduction (–2.4%)

* Margins improve across all routes, but:
* Long-haul routes remain loss-making.
* Fuel relief reduces losses but does not restore profitability.

**Executive conclusion:**
Fuel price movements are a second-order lever. Structural economics dominate long-haul outcomes.

#### Scenario B — Load Factor Improvement (83.8%)

* Highest impact scenario
* Long-haul routes benefit most due to fixed-cost dilution.
* Margins improve meaningfully without additional capacity or cost increases.

**Executive conclusion:**
Load factor optimization is the single most powerful profitability lever in the current network.

#### Scenario C — Ancillary Revenue Upside (+5.5%)

* Margins improve incrementally across all route types.
* Strongest relative impact on short- and medium-haul routes.

**Executive conclusion:**
Ancillary revenue is a margin enhancer, not a turnaround lever.

#### Strategic Implications

* Protect and grow short- & medium-haul capacity as margin stabilizers.

* Long-haul routes require structural action:
    * Pricing discipline
    * Fleet optimization
    * Frequency rationalization
* Commercial focus should prioritize utilization over cost relief.

In [418]:
# =========================
# MASTER DATAFRAME
# =========================
df = lh_fra_routes.copy()